In [1]:
import sys
# sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
sys.path.append("/s/ls4/users/cappukan/commands_rdf/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import re
from collections import defaultdict
from copy import deepcopy
import torch

torch.cuda.is_available()
import yaml
import pyhocon
import joblib
from copy import deepcopy
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from RobotCommandClassifier import utils
from RobotCommandClassifier import *
from MultilabelML import PrepareData, PrepareInput_for_simpletransformers, PrepareModel

In [42]:
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/SimpleLM.conf")
#CONFIG = configFileContent['rubert_tiny2_multilabel_noYno0_fold4'].as_plain_ordered_dict()
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
#CONFIG = configFileContent['MyMultiTiny2_data2_fold0'].as_plain_ordered_dict()

# cfg_path = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiMLangBert_data3v2/fold_2/config.json"

cfg_path = "/s/ls4/users/cappukan/commands_rdf/CommandClassifier/models/MyMultiTiny2_data4/fold_4/config.json"
# cfg_path = "/s/ls4/users/cappukan/commands_rdf/CommandClassifier/models/MyMultiTiny2_data4/fold_3/config.json"
# cfg_path = "/s/ls4/users/cappukan/commands_rdf/CommandClassifier/models/MyMultiTiny2_data4/fold_2/config.json"
# cfg_path = "/s/ls4/users/cappukan/commands_rdf/CommandClassifier/models/MyMultiTiny2_data4/fold_1/config.json"
# cfg_path = "/s/ls4/users/cappukan/commands_rdf/CommandClassifier/models/MyMultiTiny2_data4/fold_0/config.json"

with open(cfg_path, "r") as f:
    CONFIG = json.load(f)

# если хотим протестировать на всем тесте, включая фолды, на которых он обучался
#CONFIG["Data"].pop("test_only_on_fold")

In [43]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = PrepareData(CONFIG)

In [44]:
sorted([x for x in os.listdir(CONFIG["output_dir"] + '/models/') if "epoch" in x])

['checkpoint-103512-epoch-19',
 'checkpoint-10896-epoch-2',
 'checkpoint-108960-epoch-20',
 'checkpoint-16344-epoch-3',
 'checkpoint-21792-epoch-4',
 'checkpoint-27240-epoch-5',
 'checkpoint-32688-epoch-6',
 'checkpoint-38136-epoch-7',
 'checkpoint-43584-epoch-8',
 'checkpoint-49032-epoch-9',
 'checkpoint-5448-epoch-1',
 'checkpoint-54480-epoch-10',
 'checkpoint-59928-epoch-11',
 'checkpoint-65376-epoch-12',
 'checkpoint-70824-epoch-13',
 'checkpoint-76272-epoch-14',
 'checkpoint-81720-epoch-15',
 'checkpoint-87168-epoch-16',
 'checkpoint-92616-epoch-17',
 'checkpoint-98064-epoch-18']

In [1]:
epoch_i = 20
# MyMultiTiny2
epoch_to_checkpoint = {1:2058,2:4116,3:6174,4:8232,5:10290,6:12348,7:14406,8:16464,9:18522,10:20580}
# MyMultiTiny2_data2
epoch_to_checkpoint = {1:6378,2:12756,3:19134,4:25512,5:31890,6:38268,7:44646,8:51024,9:57402,10:63780}
#MyMultiTiny2_data3
epoch_to_checkpoint = {1:5481,2:10962,3:16443,4:21924,5:27405,6:32886,7:38367,8:43848,9:49329,10:54810,
                      11:60291, 12:65772, 13:71253, 14:76734, 15:82215, 16:87696, 17:93177, 18:98658, 19:104139, 20:109620}
#MyMultiTiny2_data3 fold 3, 4
epoch_to_checkpoint = {1:5482,2:10964,3:16446,4:21928,5:27410,6:32892,7:38374,8:43856,9:49338,10:54820,
                      11:60302, 12:65784, 13:71266, 14:76748, 15:82230, 16:87712, 17:93194, 18:98676, 19:104158, 20:109640}

#MyMultiTiny2_data4 fold 0, 1, 2, 3, 4
epoch_to_checkpoint = {1:5448, 2:10896, 3:16344, 4:21792, 5:27240, 6:32688, 7:38136, 8:43584, 9:49032, 10:54480,
                       11:59928, 12:65376, 13:70824, 14:76272, 15:81720, 16:87168, 17:92616, 18:98064, 
                       19:103512, 20:108960}

CONFIG["Model"]['model_name'] = CONFIG["output_dir"] + '/models/checkpoint-{}-epoch-{}'.format(epoch_to_checkpoint[epoch_i], epoch_i)

model = PrepareModel(CONFIG)

NameError: name 'CONFIG' is not defined

# Make prediction

In [46]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())
#predictions = np.array(predictions)

  0%|          | 0/263 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [47]:
#привести бинарный мультилейбл к мультиклассовому
classes_per_attribute = CONFIG["Model"]['num_sublabels_per_biglabel']
predictions_2 = utils.binarymultilabel_to_multiclassmultilabel(raw_outputs, classes_per_attribute, CONFIG)

In [48]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_classes_report.json".format(epoch_i)), "w") as f:
    json.dump(result, f)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_avg_report.json".format(epoch_i)), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       0.67      0.20      0.31        10
         1.0       0.00      0.00      0.00         0
         2.0       0.56      0.62      0.59         8
         3.0       0.86      0.60      0.71        10
         6.0       0.98      0.99      0.99       235
         7.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0

    accuracy                           0.94       263
   macro avg       0.38      0.30      0.32       263
weighted avg       0.95      0.94      0.94       263

direction
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       245
         1.0       0.67      0.40      0.50         5
         2.0       1.00      0.33      0.50         3
         3.0       0.00      0.00      0.00         0
         4.0       1.00      0.80      0.89        10
       

/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

78.0 	 correct_samples_perc
94.0 	 [action]_acc
32.0 	 [action]_macrof1
97.0 	 [direction]_acc
48.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
94.0 	 [object1]_acc
81.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
97.0 	 [relation1]_acc
97.0 	 [relation1]_macrof1
92.0 	 [object2]_acc
92.0 	 [object2]_macrof1
95.0 	 [relation2]_acc
95.0 	 [relation2]_macrof1
91.0 	 [object3]_acc
91.0 	 [object3]_macrof1
100.0 	 [self]_acc
100.0 	 [self]_macrof1
98.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
78.0 	 avg_macro_f1
96.0 	 avg_acc


/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [49]:
possible_combinations = pd.read_csv("../Data/Interim/possible_combinations_of_attributes_for_actions.csv")
possible_combinations_arr = possible_combinations.loc[:, CONFIG["Data"]["target_columns"]].values

In [50]:
# Rule
def softmax(x):
    #e_x = x - np.expand_dims(np.max(x, axis=1), axis=1)
    #return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)
    e_x = np.exp(x)
    return e_x / np.expand_dims(np.sum(e_x, axis=1), axis=1)
softmax_outputs = np.zeros_like(raw_outputs, dtype=np.float32)
shift = 0
for num_sublabels in CONFIG['Model']['num_sublabels_per_biglabel']:
    softmax_outputs[:, shift:shift + num_sublabels] = softmax(raw_outputs[:, shift:shift + num_sublabels])
    shift += num_sublabels

predictions_3 = []
for i in range(len(softmax_outputs)):
    shift = 0
    probs_for_combinations = np.zeros_like(possible_combinations_arr, dtype=np.float32)
    # предполагается, что первые значения в выходном векторе из модели - классы экшена
    assert CONFIG["Data"]["target_columns"][0]=="action"
    probs_for_combinations[:,0] = np.take(softmax_outputs[i, shift:shift+classes_per_attribute[0]], possible_combinations_arr[:,0])
    shift += classes_per_attribute[0]
    #print(probs_for_combinations[:,0])
    maxprob_attribute_classes = [-1] # -1 for action
    for attribute_i in range(1, len(classes_per_attribute)):
        # есть вариант ставить 0 для нулевых классов или наоборот - обратное от максимального класса
        probs = softmax_outputs[i, shift:shift+classes_per_attribute[attribute_i]]
        assert np.round(sum(probs), 5)==1
        zerocls_prob = probs[0]
        nonzerocls_max_prob = np.max(probs[1:])
        #print(zerocls_prob, nonzerocls_max_prob)
        maxprob_attribute_classes.append(np.argmax(probs[1:])+1)
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==0, attribute_i] = zerocls_prob
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==1, attribute_i] = nonzerocls_max_prob
        shift += classes_per_attribute[attribute_i]

    best_template_i = np.argmax(np.sum(probs_for_combinations, axis=1))
    sample_prediction = possible_combinations_arr[best_template_i].copy()
    for i in range(1, len(classes_per_attribute)):
        if sample_prediction[i]!=0:
            sample_prediction[i]=maxprob_attribute_classes[i]
    predictions_3.append(sample_prediction)
predictions_3 = np.array(predictions_3)

In [51]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
result = utils.calculate_metrics_2(test_y_df, predictions_3, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_classes_report_rule.json".format(epoch_i)), "w") as f:
    json.dump(result, f)

result_avg = utils.calculate_metrics(test_y_df, predictions_3, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_avg_report_rule.json".format(epoch_i)), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

           0       0.80      0.40      0.53        10
           1       0.00      0.00      0.00         0
           2       0.56      0.62      0.59         8
           3       1.00      0.60      0.75        10
           6       0.98      0.99      0.99       235
           7       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0

    accuracy                           0.94       263
   macro avg       0.42      0.33      0.36       263
weighted avg       0.96      0.94      0.95       263

direction
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       245
           1       0.67      0.40      0.50         5
           2       1.00      0.33      0.50         3
           3       0.00      0.00      0.00         0
           4       1.00      0.80      0.89        10
       

/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

79.0 	 correct_samples_perc
94.0 	 [action]_acc
36.0 	 [action]_macrof1
97.0 	 [direction]_acc
48.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
93.0 	 [object1]_acc
80.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
97.0 	 [relation1]_acc
97.0 	 [relation1]_macrof1
92.0 	 [object2]_acc
92.0 	 [object2]_macrof1
94.0 	 [relation2]_acc
94.0 	 [relation2]_macrof1
92.0 	 [object3]_acc
91.0 	 [object3]_macrof1
100.0 	 [self]_acc
100.0 	 [self]_macrof1
98.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
78.0 	 avg_macro_f1
96.0 	 avg_acc


/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/cappukan/anaconda3/envs/rdfgen/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r